<a href="https://colab.research.google.com/github/AchmInvest/TestIAPublic/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar datos desde Google Drive

In [3]:
import numpy as np
import scipy as sc
import pandas as pd
from google.colab import drive
 
drive.mount('/content/gdrive')
#una vez montado tenemos acceso a los ficheros
# de Drive bajo el Path /content/gdrive/My Drive
Data=pd.read_csv('/content/gdrive/My Drive/Machine learning/datatestpanda.csv')



Mounted at /content/gdrive


**Lectura de datos de los sats** 


In [4]:
from random import random

sats_list=["E01", "E13", "E15", "E21", "E26", "E27", "G25", "G26", "G29", "G31"]
epoch_data_dict = {'epoch' : {'sat': []}}
SCALE_FACTOR = 25e6
SCALE_FACTOR_EPOCH = 100
OFFSET_SATS=25e6

for sat in sats_list:
  filename=str(sat+"_data.txt")
  print (filename)
  path= str("/content/gdrive/My Drive/Machine learning/"+filename) 
  file=open(path, 'r')

  for line in file:
    line_read=file.readline()
    sat_data_line= line_read.split(' ')

    if (len(sat_data_line) == 7):
      line_sat_data = []
      line_sat_data.append(sat_data_line[0])        # SatID
      line_sat_data.append(  float(sat_data_line[1]) / SCALE_FACTOR / SCALE_FACTOR_EPOCH) # Epoch
      line_sat_data.append( (float(sat_data_line[2]) + OFFSET_SATS) / SCALE_FACTOR ) # CodeM
      line_sat_data.append( (float(sat_data_line[3]) + OFFSET_SATS) / SCALE_FACTOR ) # XECEF
      line_sat_data.append( (float(sat_data_line[4]) + OFFSET_SATS) / SCALE_FACTOR ) # YECEF
      line_sat_data.append( (float(sat_data_line[5]) + OFFSET_SATS) / SCALE_FACTOR ) # ZECEF
      line_sat_data.append( (float(sat_data_line[6]) + OFFSET_SATS) / SCALE_FACTOR ) # CLKSV

      key = round(line_sat_data[1], 12)
      if key not in epoch_data_dict.keys():
        epoch_data_dict[key] = {}

      epoch_data_dict[key][line_sat_data[0]] = line_sat_data
  

E01_data.txt
E13_data.txt
E15_data.txt
E21_data.txt
E26_data.txt
E27_data.txt
G25_data.txt
G26_data.txt
G29_data.txt
G31_data.txt


**Lectura de datos de la GSS**

In [5]:
gss_data_filename='gss_results_epoch.txt'
gss_path= str("/content/gdrive/My Drive/Machine learning/"+gss_data_filename) 

file_gss=open(gss_path, 'r')

gss_data_dict = {'epoch' : []}

for line in file_gss:
#for i in range(10):
  line_read=file_gss.readline()
  gss_data_line= line_read.split(' ')

  if sat_data_line[0] == "#":
    continue

  if (len(gss_data_line) == 5):
      line_gss_data = []
      line_gss_data.append(  float(gss_data_line[0]) / SCALE_FACTOR / SCALE_FACTOR_EPOCH) # Epoch
      line_gss_data.append( (float(gss_data_line[1]) + OFFSET_SATS) /SCALE_FACTOR) # XGSS
      line_gss_data.append( (float(gss_data_line[2]) + OFFSET_SATS) /SCALE_FACTOR) # YGSS
      line_gss_data.append( (float(gss_data_line[3]) + OFFSET_SATS) /SCALE_FACTOR) # ZGSS
      line_gss_data.append( (float(gss_data_line[4]) + OFFSET_SATS) /SCALE_FACTOR) # CLKGSS
      
      key = round(line_gss_data[0], 12)
      if key not in epoch_data_dict.keys():
        gss_data_dict[key] = {}

      gss_data_dict[key] = line_gss_data 
  


**Unificacion de datos de sats y gss para facilitar el entrenamiento**

In [11]:
#nuevo mapa que tenga los datos de los sats y de las GSS para cada epoca
sats_and_gss_data={'epoch':{'sat_gss':[]}} # guardamos el array de la gss como en el mismo mapa que los sats con la key GSS

# recorremos las epocas para las que tenemos datos de la gss y vamos añadiendo esos datos y los de los sats en el array unificado
for keys in gss_data_dict.keys():
  if keys in epoch_data_dict.keys():
    #print("Epoca coincidente!")
    #incializamos el dict unificado para esta epoca
    if keys not in sats_and_gss_data.keys():
      sats_and_gss_data[keys] = {}
      sats_and_gss_data[keys] = epoch_data_dict[keys] #cogemos todos los sats
      sats_and_gss_data[keys]['GSS'] = gss_data_dict[keys] #cogemos los datos de la gss bajo la key GSS


# vamos a crear una funcion que dada una key nos de dos arrays, uno con los datos de los satélites de forma consecutiva y otro array con los datos de la gss
def GetTrainingDataForEpoch(epoch, data):
  SatsData=[]
  GSS_ref=[]

  if epoch in data.keys():
    if epoch != 'epoch': #Para no contar el elemento de inicializacion del mapa
      SatsData.append(epoch) # añadimos la epoca como el primer parámetro del vector de entrenamiento
      for key in data[epoch].keys():
        if key == 'GSS':
          GSS_ref= data[epoch][key]
        elif len(data[epoch][key]) == 7:
          SatsData.append(data[epoch][key][2]) # apendeamos medida de código
          SatsData.append(data[epoch][key][3]) # apendeamos Xsat
          SatsData.append(data[epoch][key][4]) # apendeamos Ysat
          SatsData.append(data[epoch][key][5]) # apendeamos Zsat
          SatsData.append(data[epoch][key][6]) # apendeamos clksat

  return SatsData, GSS_ref



**Introduccion Tensor Flow** 
Definición de la red neuronal
1.   Definicion de las variables auxiliares de configuracion de las capas
1.   Definicion de los placeholders de entrada y salida de datos de la red neuronal
2.   Definicion de las matrices de pesos y biases de las capas intermedias
2.   Definicion de la funcion de coste
3.   Definicion del optimizador que se va a usar

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import math
from matplotlib import animation
from IPython.core.display import display, HTML

#print all data 
#for key2 in epoch_data_dict.keys():
#  print(" length epoch dict" +str(len(epoch_data_dict)))
#  print(" length sat dict: " + str(len(epoch_data_dict[key2])))
#  for key3 in epoch_data_dict[key2].keys():
#    data = epoch_data_dict[key2][key3]
#    print(key2, " ", key3, data)

# Definimos los puntos de entrada de la red (neuronas de entrada)
# - vamos a plantear un numero de neuronas para cada satélite de 5
# - una neurona extra para la epoca de entrada
# - 5 neuronas de salida, que serán las posiciones XYZ ECEF, el reloj de la estacion y la epoca
num_sats=10
num_neur_per_sat=5
total_input_neur= num_sats*num_neur_per_sat+1 #1 neur para la epoca
output_neur=5

lr = 5e-11           # learning rate
neuronas_por_capa = [total_input_neur, math.floor(total_input_neur*1.8), math.floor(total_input_neur*1.2), math.floor(total_input_neur*0.5), output_neur]  # número de neuronas por capa.
print(neuronas_por_capa)

#iepoch = tf.placeholder('double', shape=[None, X.shape[1]])
#iY = tf.placeholder('double', shape=[None])
iSatsData = tf.placeholder('float', shape=[1,total_input_neur])
GSS_ref = tf.placeholder('float', shape=[1,5])

# Capa 1
initW1 = np.ones((neuronas_por_capa[0], neuronas_por_capa[1]), 'float32')
initb1 = np.ones(neuronas_por_capa[1], 'float32')
#W1 = tf.Variable(tf.random_normal([neuronas_por_capa[0], neuronas_por_capa[1]]), name='Weights_1')
W1 = tf.Variable(initW1, name='Weights_1')
b1 = tf.Variable(initb1, name='bias_1')
l1 = tf.nn.relu(tf.add(tf.matmul(iSatsData, W1), b1))

# Capa 2
initW2 = np.ones((neuronas_por_capa[1], neuronas_por_capa[2]), 'float32')
initb2 = np.ones(neuronas_por_capa[2], 'float32')
W2 = tf.Variable(initW2, name='Weights_2')
#b2 = tf.Variable(tf.random_normal([neuronas_por_capa[2]]), name='bias_2')
b2 = tf.Variable(initb2, name='bias_2')
l2 = tf.nn.relu(tf.add(tf.matmul(l1, W2), b2))

# Capa 3
initW3 = np.ones((neuronas_por_capa[2], neuronas_por_capa[3]), 'float32')
initb3 = np.ones(neuronas_por_capa[3], 'float32')
#W3 = tf.Variable(tf.random_normal([neuronas_por_capa[2], neuronas_por_capa[3]]), name='Weights_3')
W3 = tf.Variable(initW3, name='Weights_3')
#b3 = tf.Variable(tf.random_normal([neuronas_por_capa[3]]), name='bias_3')
b3 = tf.Variable(initb3, name='bias_3')
l3 = tf.nn.relu(tf.add(tf.matmul(l2, W3), b3))

# Capa 3
initW4 = np.ones((neuronas_por_capa[3], neuronas_por_capa[4]), 'float32')
initb4 = np.ones(neuronas_por_capa[4], 'float32')
#W3 = tf.Variable(tf.random_normal([neuronas_por_capa[2], neuronas_por_capa[3]]), name='Weights_3')
W4 = tf.Variable(initW3, name='Weights_4')
#b3 = tf.Variable(tf.random_normal([neuronas_por_capa[3]]), name='bias_3')
b4 = tf.Variable(initb3, name='bias_4')

# Vector de predicciones de Y.
OutputEpoch = tf.nn.relu(tf.add(tf.matmul(l3, W4), b4))

# Evaluación de las predicciones.
loss = tf.losses.mean_squared_error(OutputEpoch, GSS_ref)

# Definimos al optimizador de la red, para que minimice el error.
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(loss)

n_steps = 1000 # Número de ciclos de entrenamiento.

iPY = [] # Aquí guardaremos la evolución de las predicción, para la animación.


[51, 91, 61, 25, 5]


RuntimeError: ignored

**Entrenamiento de la red neuronal y optimizacion de la funcion de coste**

In [9]:
from numpy import array 
import sys
idx=0
# Creamos una session para trabajar
with tf.Session() as sess:
  
  # Inicializamos todos los parámetros de la red, las matrices W y b.
  sess.run(tf.global_variables_initializer())
    
  # Iteramos n pases de entrenamiento.
  #for step in range(n_steps):
  # Iteramos el entrenamiento para cada epoca de las que tenemos datos:
  for epoch in sats_and_gss_data.keys():
    
    # Evaluamos al optimizador, a la función de coste y al tensor de salida pY. 
    # La evaluación del optimizer producirá el entrenamiento de la red.
    # Como datos de entrada vamos a ir cogiendo epocas de los mapas y usamos la funcion GetTrainingDataForEpoch para obtener ya ordenados los datos
    sats_data_training = []
    gss_ref_training = []
    sats_data_training, gss_ref_training = GetTrainingDataForEpoch(epoch, sats_and_gss_data)
    if len(sats_data_training) != 51:
      continue
    print("   Training sat data: ", sats_data_training)
    print("   Expected result: ", gss_ref_training)
    sats_data_training_reshape= array(sats_data_training).reshape(1,51)
    gss_ref_training_reshape= array(gss_ref_training).reshape(1,5)
    _, _loss, _pY = sess.run([optimizer, loss, OutputEpoch], feed_dict={ iSatsData : sats_data_training_reshape, GSS_ref : gss_ref_training_reshape })
    
    print(_pY)
    # Cada 3 iteraciones, imprimimos métricas.
    idx=idx+1
    if idx % 1 == 0: 
      # Cálculo del accuracy.
      acc = np.mean(np.round(_pY) - gss_ref_training < 1e-12)
      
      # Impresión de métricas.
      print('Step', idx, '/', n_steps, '- Loss = ', _loss, '- Acc =', acc)
      
      # Obtenemos predicciones para cada punto de nuestro mapa de predicción _pX.
      #_pY = sess.run(pY, feed_dict={ iX : _pX }).reshape((res, res))

      # Y lo guardamos para visualizar la animación.
      #iPY.append(_pY)
      if idx==2000:
        break


tf.io.write_file("W1.txt", W1)

Se han truncado las últimas 5000 líneas del flujo de salida.
   Training sat data:  [0.494165100156, 1.9549030905949198, 1.1935325310809999, 0.9874624684728, 1.1652539483000401, 0.99337577988848, 1.8208351610596, 1.19353251800764, 0.98746246865152, 1.16525394111144, 1.00459827462208, 1.8869548460290402, 1.1935325310809999, 0.9874624684728, 1.1652539483000401, 1.01101446190156, 1.82973409571168, 1.1935325310809999, 0.9874624684728, 1.1652539483000401, 0.99374121892028, 1.8687868494028002, 1.1935325310809999, 0.9874624684728, 1.1652539483000401, 1.0581742372954, 1.87156084894656, 1.1935325310809999, 0.9874624684728, 1.1652539483000401, 1.0062330645272801, 1.7985890287888, 1.1935325310809601, 0.98746246847276, 1.1652539483000401, 0.9912818212719201, 1.76972308285792, 1.1935325310809601, 0.98746246847276, 1.1652539483000401, 1.00163901382852, 1.7384370279893602, 1.1935325310809601, 0.98746246847276, 1.1652539483000401, 1.00311667828612, 1.7150887578592802, 1.1935325310809601, 0.98746246847

TypeError: ignored